# Exploring the billing datasets

```Do not run in poetry shell```

NOT TESTED AFTER REMAKE OF BRONZE

In [39]:
# Imports
import os
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [40]:
# Set variables
CONNECTION_STRING = os.environ.get("STORAGE_CONNECTION_STRING")
CONTAINERNAME = os.environ.get("STORAGE_CONTAINER_NAME_1")
BLOBNAME = os.environ.get("STORAGE_BLOB_NAME")

In [41]:
def create_sas():
    CONTAINERNAME = os.environ.get("STORAGE_CONTAINER_NAME_1")
    BLOBNAME = os.environ.get("STORAGE_BLOB_NAME")
    ACCOUNTNAME = os.environ.get("STORAGE_ACCOUNT_NAME")
    QUERYSTRING = os.environ.get("BLOB_QUERY_STRING")
    
    sas_url = f"https://{ACCOUNTNAME}.blob.core.windows.net/{CONTAINERNAME}/cloudlink/azure/{BLOBNAME}{QUERYSTRING}"
    return sas_url
 
    
sas_url = create_sas()

In [42]:
df = pd.read_parquet(sas_url)
df_len, df_width = df.shape
print('Rows: ' +str(df_len),'\nColumns: ' + str(df_width))
print(df.columns)

Rows: 1755 
Columns: 65
Index(['invoiceId', 'previousInvoiceId', 'billingAccountId',
       'billingAccountName', 'billingProfileId', 'billingProfileName',
       'invoiceSectionId', 'invoiceSectionName', 'resellerName',
       'resellerMpnId', 'costCenter', 'billingPeriodEndDate',
       'billingPeriodStartDate', 'servicePeriodEndDate',
       'servicePeriodStartDate', 'date', 'serviceFamily', 'productOrderId',
       'productOrderName', 'consumedService', 'meterId', 'meterName',
       'meterCategory', 'meterSubCategory', 'meterRegion', 'ProductId',
       'ProductName', 'SubscriptionId', 'subscriptionName', 'publisherType',
       'publisherId', 'publisherName', 'resourceGroupName', 'ResourceId',
       'resourceLocation', 'location', 'effectivePrice', 'quantity',
       'unitOfMeasure', 'chargeType', 'billingCurrency', 'pricingCurrency',
       'costInBillingCurrency', 'costInPricingCurrency', 'costInUsd',
       'paygCostInBillingCurrency', 'paygCostInUsd',
       'exchangeRatePri

In [ ]:
# clean az data
df = df.replace(r'\s+',np.nan,regex=True).replace('',np.nan)
before = len(df.columns)
# Drop all columns where all values is NaN
df = df.dropna(axis=1,how='all')
after = len(df.columns)
removed = before-after
print("Removed "+ str(removed) + " blank columns!")

In [ ]:
df.columns

In [ ]:
for column in df:
    x = df
    x = x[column].drop_duplicates().dropna()
    print("The column '" + column + "' has " + str(x.shape[0]) + " unique rows")

In [ ]:
subscription_colums = ["SubscriptionId","subscriptionName", "billingAccountId", "billingAccountName", "billingProfileId", "billingProfileName"]

subscription_dim = pd.DataFrame(df,columns=subscription_colums, copy=True)
subscription_dim.index = subscription_dim["SubscriptionId"]
subscription_dim.drop_duplicates("SubscriptionId", inplace=True)
subscription_dim.columns


In [ ]:
product_colums = ["ProductId","ProductName", "resourceLocation", "meterSubCategory", "meterCategory"]

product_dim = pd.DataFrame(df,columns=product_colums, copy=True)
product_dim.index = product_dim["ProductId"]
product_dim.drop_duplicates("ProductId", inplace=True)
product_dim.columns

In [ ]:
resource_dim = pd.DataFrame(df,columns=["ResourceId"], copy=True)
resource_dim = resource_dim['ResourceId'].str.split("/", expand=True)
resource_dim = resource_dim.loc[:, [2,4,6,8]]
resource_dim.columns = ['SubscriptionId', 'resourceGroupName', 'consumedService', 'resourceName']
resource_dim.index = resource_dim["resourceName"]
resource_dim.drop_duplicates("resourceName", inplace=True)
resource_dim.columns

In [44]:
tags_colums = ['ResourceId', 'tags']

tags_dim = pd.DataFrame(df,columns=tags_colums, copy=True)
tags_dim.index = tags_dim['ResourceId']

tags_dim.columns

Index(['ResourceId', 'tags'], dtype='object')

In [71]:
def find_longest():
    longest = len(tags_dim['tags'][0])
    for elem in tags_dim['tags']:
        if elem is None:
            continue
        
        next = len(elem)
        if  next <= longest:
            continue
        
        else:
            longest = next
            longest_element = elem
    return longest_element
    
longest_tag = find_longest()
print(len(longest_tag), longest_tag)

115
180 {  "application": "databricks",  "databricks-environment": "true",  "environment": "dev",  "location": "norway",  "datalayer": "raw",  "datatier": "bronze",  "sensitivity": "none"}


In [ ]:
usage_colums = ["quantity", "effectivePrice", "paygCostInBillingCurrency", "date", "ProductId", "SubscriptionId"]

usage_fact = pd.DataFrame(df,columns=usage_colums, copy=True)
usage_fact.index = usage_fact["ProductId"]
usage_fact = usage_fact[usage_fact['effectivePrice'] != 0]
usage_fact.columns